Code for TensorBoard
===

In [1]:
!rm -rf logs/
!rm ngrok*
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2018-08-17 08:17:57--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.44.174.39, 52.86.186.182, 54.152.208.69, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.44.174.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  8.60MB/s    in 0.6s    

2018-08-17 08:17:58 (8.60 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = 'logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [4]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://cf190af4.ngrok.io


Code for Downloading data from Google Drive
===

In [5]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1wlgaPV2J-An9K_TiNFz8iwyaZiaDwArX' in parents"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)

title: reviews.txt, id: 1aij3k6M-g3Kpt3krlDyn_5PjRuxj40Tz
downloading to /content/data/reviews.txt
title: labels.txt, id: 1tsf7CXZpb6MFFu6xf6oYMAJYUM0qecnv
downloading to /content/data/labels.txt


Preparing the Dataset
===

In [0]:
import numpy as np

In [0]:
with open('./data/reviews.txt', 'r') as file:
  features = file.read()

In [0]:
features = features.split('\n')

## Getting trigrams

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
def vectorizer(text):
  vec = CountVectorizer(binary=True, min_df=3, ngram_range=(3, 3), token_pattern='(?u)\\b[a-z0-9\-\_]{3,}\\b', stop_words='english')
  vec.fit(text)
  return vec

In [0]:
vec = vectorizer(features)
ngrams = vec.transform(features).toarray()

In [12]:
ngrams

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [13]:
ngrams.shape

(25000, 15146)

## One-hot encoding the labels

In [0]:
with open('./data/labels.txt', 'r') as file:
  labels = file.read()

In [0]:
labels = np.array([1 if label == 'positive' else 0 for label in labels.split('\n')])

In [16]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
labels = to_categorical(labels)

## Splitting the dataset

In [0]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(ngrams, labels[0:25000], test_size=0.30, stratify=labels[0:25000])

Building the Model
===

In [0]:
import tensorflow as tf

In [0]:
tf.reset_default_graph()

with tf.name_scope('input'):
  input_features = tf.placeholder(dtype=tf.float32, shape=[None, ngrams.shape[1]],
                                 name='input_features')
  input_labels = tf.placeholder(dtype=tf.float32, shape=[None, labels.shape[1]],
                               name='input_labels')

  with tf.name_scope('model'):
    weights = {
        'hidden_layer': tf.Variable(tf.random_normal(shape=[ngrams.shape[1], 16]),
                          name='weights'),
        'output_layer': tf.Variable(tf.random_normal(shape=[16, labels.shape[1]]),
                          name='weights'),
    }
    
    biases = {
        'hidden_layer': tf.Variable(tf.random_normal(shape=[16])),
        'output_layer': tf.Variable(tf.random_normal(shape=[labels.shape[1]])),
    }
    
    hidden_layer = tf.add(tf.matmul(input_features, weights['hidden_layer']),
                         biases['hidden_layer'])
    hidden_layer = tf.nn.relu(hidden_layer)
    output_layer = tf.add(tf.matmul(hidden_layer, weights['output_layer']),
                         biases['output_layer'])
    predictions = tf.nn.softmax(output_layer)
    
with tf.name_scope('training_ops'):
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
      logits=output_layer, labels=input_labels))
  train_op = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(loss)
  tf.summary.scalar(name='loss', tensor=loss)

with tf.name_scope('metrics'):
  correct_prediction = tf.cast(tf.equal(tf.argmax(predictions, 1),
                                         tf.argmax(input_labels, 1)), tf.float32)
  accuracy_op = tf.reduce_mean(correct_prediction)
  tf.summary.scalar(name='accuracy', tensor=accuracy_op)

summary = tf.summary.merge_all()
writer = tf.summary.FileWriter(logdir='logs', graph=tf.get_default_graph())

In [0]:
init_op = tf.global_variables_initializer()

In [0]:
def next_batch(batch_size, features, labels):
  indices = np.arange(start=0, stop=features.shape[0])
  np.random.shuffle(indices)
  indices = indices[:batch_size]
  return features[indices], labels[indices]

In [23]:
with tf.Session() as sess:
  sess.run(init_op)
  
  # epochs = 10
  for epoch in range(10):
    for mini_batch in range(int(x_train.shape[0] / 16)):
      batch_x, batch_y = next_batch(batch_size=16, features=x_train, labels=y_train)
      
      _, train_loss, train_accuracy, train_summary = sess.run([train_op, loss,
                                                               accuracy_op,
                                                              summary],
                                              feed_dict={input_features: batch_x,
                                                        input_labels: batch_y})
      writer.add_summary(summary=train_summary, global_step=mini_batch)
    print('Epoch {}, loss : {}, accuracy : {}'.format(epoch, train_loss,
                                                      train_accuracy))  
  
  test_accuracy = sess.run(accuracy_op, feed_dict={input_features: x_test, input_labels: y_test})
  print('Test accuracy : {}'.format(test_accuracy))

Epoch 0, loss : 0.5302518606185913, accuracy : 0.625
Epoch 1, loss : 0.20831575989723206, accuracy : 0.875
Epoch 2, loss : 0.4056752622127533, accuracy : 0.6875
Epoch 3, loss : 0.20885398983955383, accuracy : 0.875
Epoch 4, loss : 0.22582387924194336, accuracy : 0.8125
Epoch 5, loss : 0.17218583822250366, accuracy : 0.9375
Epoch 6, loss : 0.06256500631570816, accuracy : 0.9375
Epoch 7, loss : 0.16024157404899597, accuracy : 0.875
Epoch 8, loss : 0.12726999819278717, accuracy : 0.9375
Epoch 9, loss : 0.08074752986431122, accuracy : 1.0
Test accuracy : 0.6622666716575623


In [24]:
features[100]

'i find it so amazing that even after all these years  we are still talking about this movie  obviously this movie wasn  t that bad or else people wouldn  t even bother to talk about it . i personally enjoyed this film immensly  and still do  i guess this film isn  t for everyone  but it certainly did touch the hearts of many .  br    br   as for those that think that this film is  overrated  or  over  hyped  . . . well  we only have the movie  going public to thank for that  lol you see  it  s not critics  article writers that make a film  huge  or a  hit  with the general movie  going public . people make the film a huge success . with titanic  everyone was in awe . let  s face it  a film like this had never been made before . at least not with the type of special effects needed to really capture the essence of the ship actually sinking . this film is so accurate that even james cameron timed the actual sinking of the ship in the film with the real sinking that fateful day in april  

In [25]:
labels[100]

array([0., 1.], dtype=float32)

In [26]:
labels = ['Negative', 'Positive']
with tf.Session() as sess:
  sess.run(init_op)
  prediction_ = sess.run(predictions, feed_dict={input_features: x_test[1].reshape(-1, x_test.shape[1])})
  print(prediction_)
  print('predicted class : {}, true class : {}'.format(sess.run(tf.argmax(prediction_, 1)), labels[sess.run(tf.argmax(y_test[1]))]))

[[0.01973421 0.9802658 ]]
predicted class : [1], true class : Negative


In [0]:
ngrams_example = vec.transform(['the food was extremely bad!']).toarray()

In [28]:
labels = ['Negative', 'Positive']
with tf.Session() as sess:
  sess.run(init_op)
  prediction_ = sess.run(predictions, feed_dict={input_features: ngrams_example.reshape(-1, ngrams_example.shape[1])})
  print(prediction_)
  print('predicted class : {}'.format(sess.run(tf.argmax(prediction_, 1))))

[[0.9956768 0.0043232]]
predicted class : [0]
